In [1]:
import csv
import sys
import os
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
import copy
import random

sys.path.append('./backbones/SSTDA')

from model import MultiStageModel
from train import Trainer
from predict import *
from batch_gen import BatchGenerator

from src.utils import load_meta, eval_txts
from src.predict import predict_backbone
import configs.sstda_config as cfg

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic=True

In [3]:
device = 'cuda'

In [4]:
dataset = 'breakfast'     # choose from gtea, 50salads, breakfast
split = 1            # gtea : 1~4, 50salads : 1~5, breakfast : 1~4
model_name = 'sstda' # always "mstcn" in this notebook

In [5]:
cfg.iter_max_gamma = cfg.iter_max_gamma[dataset]
cfg.iter_max_beta = cfg.iter_max_beta[dataset]
cfg.iter_max_nu = cfg.iter_max_nu[dataset]
cfg.mu = cfg.mu[dataset]
cfg.eta = cfg.eta[dataset]
cfg.lr = cfg.lr[dataset]

In [6]:
actions_dict, \
num_actions, \
gt_path, \
features_path, \
vid_list_file, \
vid_list_file_tst, \
sample_rate,\
model_dir,\
result_dir, \
record_dir = load_meta(cfg.dataset_root, cfg.model_root, cfg.result_root, cfg.record_root, dataset, split, model_name)

Created :./model/sstda/breakfast/split_1
Created :./result/sstda/breakfast/split_1
Created :./record/sstda/breakfast


In [7]:
model = MultiStageModel(cfg, num_actions)

In [8]:
trainer = Trainer(num_actions)

In [9]:
batch_gen_source = BatchGenerator(num_actions, actions_dict, gt_path, features_path, sample_rate)
batch_gen_target = BatchGenerator(num_actions, actions_dict, gt_path, features_path, sample_rate)
batch_gen_source.read_data(vid_list_file)
batch_gen_target.read_data(vid_list_file_tst)

In [10]:
trainer.train(model, model_dir, result_dir, batch_gen_source, batch_gen_target, device, cfg)

./backbones/SSTDA/model.py:420: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  label_d = label_d_onehot.nonzero()  # e.g. [0, 0, 1, 0, 0, 0] --> [[2]]


[epoch 1]: epoch loss = 19.381184,   acc (S) = 0.165025,   acc (T) = 0.182106,   beta = (0.111762, 0.014487),   nu = 0.000456
[epoch 2]: epoch loss = 14.701088,   acc (S) = 0.295358,   acc (T) = 0.305042,   beta = (0.220839, 0.028388),   nu = 0.000912
[epoch 3]: epoch loss = 12.959331,   acc (S) = 0.381025,   acc (T) = 0.386962,   beta = (0.324659, 0.041190),   nu = 0.001368
[epoch 4]: epoch loss = 11.652328,   acc (S) = 0.487562,   acc (T) = 0.465787,   beta = (0.421203, 0.052549),   nu = 0.001825
[epoch 5]: epoch loss = 10.712873,   acc (S) = 0.558602,   acc (T) = 0.515110,   beta = (0.509061, 0.062300),   nu = 0.002281
[epoch 6]: epoch loss = 10.128035,   acc (S) = 0.586742,   acc (T) = 0.535787,   beta = (0.587454, 0.070436),   nu = 0.002737
[epoch 7]: epoch loss = 9.763716,   acc (S) = 0.612277,   acc (T) = 0.556211,   beta = (0.656181, 0.077064),   nu = 0.003193
[epoch 8]: epoch loss = 9.112892,   acc (S) = 0.665990,   acc (T) = 0.595066,   beta = (0.715510, 0.082360),   nu = 0.0

In [10]:
max_epoch = -1
max_val = 0.0
max_results = dict()

f = open(os.path.join(record_dir, 'split_{}_all.csv'.format(split)), 'w')

writer = csv.writer(f, delimiter='\t')
writer.writerow(['epoch', 'accu', 'edit', 
                 'F1@{}'.format(cfg.iou_thresholds[0]),
                 'F1@{}'.format(cfg.iou_thresholds[1]), 
                 'F1@{}'.format(cfg.iou_thresholds[2])])

for epoch in range(1, cfg.num_epochs+1):
    print('======================EPOCH {}====================='.format(epoch))
    predict_backbone(model_name, model, model_dir, result_dir, features_path, vid_list_file_tst, 
                     epoch, actions_dict, device, sample_rate)    
    results = eval_txts(cfg.dataset_root, result_dir, dataset, split, model_name)
    
    writer.writerow([epoch, '%.4f'%(results['accu']), '%.4f'%(results['edit']),
                    '%.4f'%(results['F1@%0.2f'%(cfg.iou_thresholds[0])]),
                    '%.4f'%(results['F1@%0.2f'%(cfg.iou_thresholds[1])]),
                    '%.4f'%(results['F1@%0.2f'%(cfg.iou_thresholds[2])])])

    curr_val = sum([results[k] for k in results.keys()])
    max_val = max(max_val, curr_val)

    if curr_val == max_val:
        max_epoch = epoch
        max_results = results

print('EARNED MAXIMUM PERFORMANCE IN EPOCH {}'.format(max_epoch))
print(max_results)

f.close()

======================EPOCH 1=====================


./backbones/SSTDA/model.py:420: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  label_d = label_d_onehot.nonzero()  # e.g. [0, 0, 1, 0, 0, 0] --> [[2]]


Acc: 21.3384
Edit: 44.3152
F1@0.10: 43.1585
F1@0.25: 36.4075
F1@0.50: 22.9656
======================EPOCH 2=====================
Acc: 32.1614
Edit: 40.8044
F1@0.10: 38.4522
F1@0.25: 30.4232
F1@0.50: 20.0242
======================EPOCH 3=====================
Acc: 42.7045
Edit: 49.0174
F1@0.10: 49.1426
F1@0.25: 41.7712
F1@0.50: 28.4617
======================EPOCH 4=====================
Acc: 48.0701
Edit: 53.7376
F1@0.10: 53.2919
F1@0.25: 46.4489
F1@0.50: 32.6076
======================EPOCH 5=====================
Acc: 50.1891
Edit: 51.7382
F1@0.10: 49.8558
F1@0.25: 42.8834
F1@0.50: 28.3093
======================EPOCH 6=====================
Acc: 43.0422
Edit: 52.5433
F1@0.10: 49.9015
F1@0.25: 41.8565
F1@0.50: 27.1730
======================EPOCH 7=====================
Acc: 60.8881
Edit: 60.8833
F1@0.10: 60.5930
F1@0.25: 55.4178
F1@0.50: 42.1563
======================EPOCH 8=====================
Acc: 58.0422
Edit: 57.2808
F1@0.10: 57.4150
F1@0.25: 50.3946
F1@0.50: 37.0068
===================

In [11]:
f = open(os.path.join(record_dir, 'split_{}_best.csv'.format(split)), 'w')
writer = csv.writer(f, delimiter='\t')
writer.writerow(['epoch', 'accu', 'edit', 
                 'F1@{}'.format(cfg.iou_thresholds[0]),
                 'F1@{}'.format(cfg.iou_thresholds[1]), 
                 'F1@{}'.format(cfg.iou_thresholds[2])])
writer.writerow([max_epoch, '%.4f'%(max_results['accu']), '%.4f'%(max_results['edit']),
                '%.4f'%(max_results['F1@%0.2f'%(cfg.iou_thresholds[0])]),
                '%.4f'%(max_results['F1@%0.2f'%(cfg.iou_thresholds[1])]),
                '%.4f'%(max_results['F1@%0.2f'%(cfg.iou_thresholds[2])])])
f.close()